In [34]:
import os

# Ruta personalizada para almacenar modelos preentrenados
custom_torch_home = r'C:\Users\User\Desktop\UAB\2nd year\2nd semester\neural network-deep learning\project\resnet_pretrained'

# Configurar la variable de entorno TORCH_HOME
os.environ['TORCH_HOME'] = custom_torch_home

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import transforms, datasets
import os
import copy
import time

In [36]:
# Ruta de la carpeta principal
main_folder = r'C:\Users\User\Desktop\UAB\2nd year\2nd semester\neural network-deep learning\project'

# ResNet input size
input_size = 224

In [37]:
# Define the Attention module
class Attention(nn.Module):
    def __init__(self, in_features, hidden_features):
        super(Attention, self).__init__()
        self.query = nn.Linear(in_features, hidden_features)
        self.key = nn.Linear(in_features, hidden_features)
        self.value = nn.Linear(in_features, hidden_features)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        query = self.query(x)
        key = self.key(x)
        value = self.value(x)
        attention_weights = self.softmax(torch.matmul(query, key.T))
        attended_values = torch.matmul(attention_weights, value)
        return attended_values

In [38]:

"""
# Define your modified ResNet model with attention
class ResNetWithAttention(nn.Module):
    def __init__(self):
        super(ResNetWithAttention, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.fc = nn.Identity() # Quitamos la capa fully connected
        self.attention = Attention(512, 512)
        self.fc = nn.Linear(512, 1)

    def forward(self, x):
        features = self.resnet(x)

        # Average pooling to 1x1
        features = features.mean(dim=[-1, -2])

        attended_features = self.attention(features)
        output = self.fc(attended_features)

        return output.squeeze(-1)
"""

class ResNetWithAttention(nn.Module):
    def __init__(self, num_classes):
        super(ResNetWithAttention, self).__init__()
        self.base_model = models.resnet50(pretrained=True)
        self.attention = Attention(self.base_model.fc.in_features, 64)
        self.base_model.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        features = self.base_model.conv1(x)
        features = self.base_model.bn1(features)
        features = self.base_model.relu(features)
        features = self.base_model.maxpool(features)
        features = self.base_model.layer1(features)
        features = self.base_model.layer2(features)
        features = self.base_model.layer3(features)
        features = self.base_model.layer4(features)

        # Average pooling to 1x1
        features = features.mean([2, 3])

        attended_features = self.attention(features.view(features.size(0), -1))

        output = self.base_model.fc(attended_features)
        return output


In [39]:
# Just normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((input_size, input_size)), 
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

Initializing Datasets and Dataloaders...


In [40]:
# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(main_folder, x), data_transforms[x]) for x in ['train', 'val']}

# Generamos los índices para el subconjunto
subset_indices_train = torch.randperm(len(image_datasets['train']))[:int(0.001*len(image_datasets['train']))]
subset_indices_val = torch.randperm(len(image_datasets['val']))[:int(0.001*len(image_datasets['val']))]
#subset_indices_test = torch.randperm(len(image_datasets['test']))[:int(0.1*len(image_datasets['test']))]

# Creamos los subconjuntos
train_data_subset = torch.utils.data.Subset(image_datasets['train'], subset_indices_train)
val_data_subset = torch.utils.data.Subset(image_datasets['val'], subset_indices_val)
#test_data_subset = torch.utils.data.Subset(image_datasets['test'], subset_indices_test)

# Create training and validation dataloaders
dataloaders_dict = {
    'train': torch.utils.data.DataLoader(train_data_subset, batch_size=batch_size, shuffle=True, num_workers=4),
    'val': torch.utils.data.DataLoader(val_data_subset, batch_size=batch_size, shuffle=True, num_workers=4),
    #'test': torch.utils.data.DataLoader(test_data_subset, batch_size=batch_size, shuffle=True, num_workers=4),
}

In [41]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    acc_history = {"train": [], "val": []}
    losses = {"train": [], "val": []}

    # we will keep a copy of the best weights so far according to validation accuracy
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    losses[phase].append(loss.item())

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data).cpu().numpy()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            
            acc_history[phase].append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, acc_history, losses

In [42]:

# Modify your initialization function to use the ResNetWithAttention model
def initialize_model(num_classes):
    model = ResNetWithAttention(num_classes)
    input_size = 50 #224
    return model, input_size


In [43]:
num_classes = 2
model = ResNetWithAttention(num_classes)
#model, input_size = initialize_model(num_classes)


c:\Users\User\anaconda3\setup\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\User\anaconda3\setup\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [44]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Number of epochs to train for 
num_epochs = 10

optimizer_ft = optim.Adam(model.parameters(), lr=0.001)

# Train and evaluate
model, hist, losses = train_model(model, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs)

Epoch 0/9
----------
train Loss: 0.7534 Acc: 0.6680
val Loss: 0.6671 Acc: 0.7037

Epoch 1/9
----------
train Loss: 0.6263 Acc: 0.6719
val Loss: 0.5886 Acc: 0.7037

Epoch 2/9
----------
train Loss: 0.6021 Acc: 0.7109
val Loss: 0.6095 Acc: 0.7037

Epoch 3/9
----------
train Loss: 0.6188 Acc: 0.7109
val Loss: 0.6196 Acc: 0.7037

Epoch 4/9
----------
train Loss: 0.6084 Acc: 0.7109
val Loss: 0.6719 Acc: 0.7037

Epoch 5/9
----------


KeyboardInterrupt: 